In [7]:
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import torchvision
import torch
import os
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve, auc
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
import pickle
from sklearn.metrics import accuracy_score, f1_score
import torch.utils.data as data_utils
from collections import defaultdict
import gc

In [28]:
def load_and_transform_cifar100(label_noise=0.0, seed=1234):
    training_data = datasets.CIFAR100(
        root="data",
        train=True,
        download=True,
    )

    test_data = datasets.CIFAR100(
        root="data",
        train=False,
        download=True,
    )    
        
    
    if label_noise > 0:
        np.random.seed(seed)
        size = int(label_noise * len(training_data))
        noise_idx = set(np.random.choice(len(training_data), size, replace=False))
        print('injecting noise ...%s' %len(noise_idx))
        targets = []
        for idx, t in enumerate(training_data.targets):
            if idx in noise_idx:
                targets.append(np.random.randint(100))
            else:
                targets.append(t)
                
        training_data.targets = targets
    
    
        
    image_size = (128, 128)
    resnet_mean = [0.485, 0.456, 0.406]
    resnet_std = [0.229, 0.224, 0.225]

    #Creating a Transformation Object
    data_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(size=image_size),
        torchvision.transforms.ToTensor(), #Converting to tensor
        torchvision.transforms.Normalize(mean = resnet_mean,
                                        std = resnet_std) 
    ])
    training_data.transform = data_transform
    test_data.transform = data_transform
    return training_data, test_data

In [35]:
def filter_data(training_data, test_data, filter_by_coarse=False, 
                coarse_label=None, coarse_to_fine=None, valid_size=0.1, batch_size=64, data_fraction=1.0, seed=1234):
    """Filters datasets by coarse label if needed and split to train and validation subsets"""

    np.random.seed(seed)
    size = int(data_fraction * len(training_data))
    sample_idx = np.random.choice(len(training_data), size, replace=False)
    print('sampling data ...%s' %len(sample_idx))
    
    if filter_by_coarse:
        labels = coarse_to_fine[coarse_label]
        indices = [idx for idx,v in enumerate(training_data.targets) if v in labels and idx in sample_idx]
        indices_test = [idx for idx,v in enumerate(test_data.targets) if v in labels]     
    else:
        indices = sample_idx
        indices_test = [i for i in range(len(test_data.targets))]

        
    
    validate_len = int(len(indices)*valid_size)
    validate_filter_indices = random.sample(range(0,len(indices)), validate_len)
    indices_val = [indices[i] for i in validate_filter_indices]  
    indices_train = [v for i,v in enumerate(indices) if i not in validate_filter_indices]    
        
    train = data_utils.Subset(training_data, indices_train)
    validate = data_utils.Subset(training_data, indices_val)
    test = data_utils.Subset(test_data, indices_test)
    
    train_dl = DataLoader(train,  batch_size=batch_size, shuffle=True)
    validate_dl = DataLoader(validate, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test,  batch_size=batch_size, shuffle=True)
    return train_dl, validate_dl, test_dl

In [23]:
class Resnext50(torch.nn.Module):
    def __init__(self, n_classes, name):
        super().__init__()
        self.name = name
        resnet = torchvision.models.resnext50_32x4d(pretrained=True, progress=True)
        resnet.fc = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.soft = torch.nn.Softmax()

    def forward(self, x):
        return self.soft(self.base_model(x))   

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer, mode="flat", lookup=[], fine_to_coarse={}, batch_size=64):
    size = len(dataloader.dataset)
    model.train()
    train_loss = 0
    counter = 0
    with tqdm(dataloader, unit="batch") as tepoch:
        counter += 1
        for X_cpu, y_cpu in tepoch:
            if mode == "fine":
                y_cpu = torch.tensor([lookup.index(i) for i in y_cpu])
            elif mode == "coarse":
                y_cpu = torch.tensor([fine_to_coarse[int(target)] for target in y_cpu])
            X, y = X_cpu.to(device), y_cpu.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = loss_fn(outputs, y.type(torch.long))
            loss.backward() 
            optimizer.step() 
            train_loss += loss.item() 
            avg_loss = train_loss / (counter * batch_size)
            tepoch.set_postfix(loss=avg_loss)
    return avg_loss
    
    
def eval_model(dataloader, model, mode="flat", lookup=[], fine_to_coarse={}, return_probab=False):
    model.eval()
    predicts = []
    targets = []
    for counter, (images, labels) in enumerate(dataloader):
        if mode == "fine":
            labels = [lookup.index(i) for i in labels]
        elif mode == "coarse":
            labels = [fine_to_coarse[int(i)] for i in labels]
        imageGPU = images.to(device)
        
        outputs = torch.Tensor.cpu(model(imageGPU))
        predicts.append(outputs.detach().numpy())
        targets.append(labels)
        
    predicts = np.vstack(predicts)
    targets = np.hstack(targets)
    if return_probab:
        return predicts, targets
    else:
        predicts = np.argmax(predicts, axis=1)
        return accuracy_score(targets, predicts)
    


In [40]:
def train_and_validate(epochs, model, loss_fn, optimizer, train_dl, val_dl, test_dl, 
                       mode="flat", fine_to_coarse={}, coarse_label=None, lookup=[], model_path = None):
    if model_path is None:
        model_path = '/models/%s.pth' %model.name
    min_loss= 1000
    max_acc = 0
    for t in range(epochs):
        if mode == "fine":
            print(f"Coarse label {coarse_label} Epoch {t+1}\n-------------------------------")
        else:
            print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        val_acc = eval_model(val_dl, model, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        if train_loss < min_loss:
            max_acc = val_acc
            min_loss = train_loss
            torch.save(model, model_path)
        elif train_loss == min_loss and val_acc>max_acc:
            max_acc = val_acc
            min_loss = train_loss
            torch.save(model, model_path)
        print("validation acc %s and loss %s" %(val_acc, train_loss))
        test_acc = eval_model(test_dl, model, mode=mode, fine_to_coarse=fine_to_coarse, lookup=lookup)
        print("done epoch %s : test_acc %s" %(t, test_acc))
    print("Done! Saved model with validation accuracy %s and loss %s" %(max_acc, min_loss))
    return test_acc

In [41]:
acorn = 1234
torch.manual_seed(acorn)
np.random.seed(acorn)

torch.cuda.is_available()

if torch.cuda.is_available():  
    dev = "cuda" 
else:  
    dev = "cpu"  
    
device = torch.device(dev)  

batch_size = 64
valid_size = 0.1

In [42]:
training_data, test_data=load_and_transform_cifar100(seed=acorn)
train_dl, validate_dl, test_dl = filter_data(training_data, test_data, valid_size=valid_size, batch_size=batch_size, data_fraction=0.1, seed=acorn)


Files already downloaded and verified
Files already downloaded and verified
sampling data ...5000


In [43]:
# Initialize the model
learning_rate = 1e-4

epochs = 15
model = Resnext50(100, name="base_model_Resnext50_01fraction")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.to(device)
loss_fn = torch.nn.CrossEntropyLoss().to(device)


In [45]:
accuracy = train_and_validate(epochs, model, loss_fn, optimizer, train_dl, validate_dl, test_dl, 
                                                    mode="flat", 
                                                    model_path = 'models/base/%s.pth' %model.name )
accuracy

  0%|          | 0/71 [00:00<?, ?batch/s]

Epoch 1
-------------------------------


/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
100%|██████████| 71/71 [00:23<00:00,  2.99batch/s, loss=4.83] 


validation acc 0.274 and loss 4.8287646025419235


  0%|          | 0/71 [00:00<?, ?batch/s]

...

  0%|          | 0/71 [00:00<?, ?batch/s]

done epoch 13 : test_acc 0.6135
Epoch 15
-------------------------------


100%|██████████| 71/71 [00:22<00:00,  3.11batch/s, loss=4.14] 


validation acc 0.634 and loss 4.135930623859167
done epoch 14 : test_acc 0.6248
Done! Saved model with validation accuracy 0.634 and loss 4.135930623859167


0.6248

In [66]:
max_acc

0.897